<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch

torch.cuda.is_available()

start = time.time()

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.sigmoid(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
train_start = time.time()

# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

end = time.time()
print(f'Total: {end - start} sec')
print(f'Train: {end - train_start} sec')


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0625
Current benign train loss: 2.3492259979248047

Current batch: 100
Current benign train accuracy: 0.25
Current benign train loss: 5.259955406188965

Current batch: 200
Current benign train accuracy: 0.1640625
Current benign train loss: 2.2781264781951904

Current batch: 300
Current benign train accuracy: 0.234375
Current benign train loss: 2.1446478366851807

Current batch: 400
Current benign train accuracy: 0.3046875
Current benign train loss: 2.0405046939849854

Current batch: 500
Current benign train accuracy: 0.25
Current benign train loss: 2.2752797603607178

Current batch: 600
Current benign train accuracy: 0.2578125
Current benign train loss: 2.0282793045043945

Current batch: 700
Current benign train accuracy: 0.2421875
Current benign train loss: 2.13023042678833

Current batch: 800
Current benign train accuracy: 0.296875
Current benign train loss: 2.016525983810425

Current batch: 900
Current benign tra


Current batch: 800
Current benign train accuracy: 0.625
Current benign train loss: 1.1469601392745972

Current batch: 900
Current benign train accuracy: 0.65625
Current benign train loss: 1.0290660858154297

Current batch: 1000
Current benign train accuracy: 0.6640625
Current benign train loss: 1.046841025352478

Current batch: 1100
Current benign train accuracy: 0.6796875
Current benign train loss: 0.9467443227767944

Total benign train accuarcy: 62.184666666666665
Total benign train loss: 1260.392986357212

[ Test epoch: 5 ]

Test accuarcy: 34.66
Test average loss: 0.03425693490505218
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.6796875
Current benign train loss: 0.8838958144187927

Current batch: 100
Current benign train accuracy: 0.71875
Current benign train loss: 0.9818999171257019

Current batch: 200
Current benign train accuracy: 0.640625
Current benign train loss: 0.9619137048721313

Current batch: 300
Current benign train accuracy: 0.679


Current batch: 200
Current benign train accuracy: 0.6953125
Current benign train loss: 0.9318276047706604

Current batch: 300
Current benign train accuracy: 0.765625
Current benign train loss: 0.7105392217636108

Current batch: 400
Current benign train accuracy: 0.671875
Current benign train loss: 0.9131094217300415

Current batch: 500
Current benign train accuracy: 0.6875
Current benign train loss: 0.8688023686408997

Current batch: 600
Current benign train accuracy: 0.6796875
Current benign train loss: 0.8606628775596619

Current batch: 700
Current benign train accuracy: 0.59375
Current benign train loss: 1.1537410020828247

Current batch: 800
Current benign train accuracy: 0.7109375
Current benign train loss: 0.8493379354476929

Current batch: 900
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8873419761657715

Current batch: 1000
Current benign train accuracy: 0.6328125
Current benign train loss: 0.9816259145736694

Current batch: 1100
Current benign train a


Current batch: 1000
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8085516095161438

Current batch: 1100
Current benign train accuracy: 0.8203125
Current benign train loss: 0.6751338243484497

Total benign train accuarcy: 72.52133333333333
Total benign train loss: 935.462976694107

[ Test epoch: 16 ]

Test accuarcy: 33.87
Test average loss: 0.027510344982147217
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.8046875
Current benign train loss: 0.6120865345001221

Current batch: 100
Current benign train accuracy: 0.7109375
Current benign train loss: 0.8232333660125732

Current batch: 200
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7495113611221313

Current batch: 300
Current benign train accuracy: 0.75
Current benign train loss: 0.8463245034217834

Current batch: 400
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7858662605285645

Current batch: 500
Current benign train accuracy:


Current batch: 400
Current benign train accuracy: 0.71875
Current benign train loss: 0.7996648550033569

Current batch: 500
Current benign train accuracy: 0.75
Current benign train loss: 0.7240889072418213

Current batch: 600
Current benign train accuracy: 0.78125
Current benign train loss: 0.6624529361724854

Current batch: 700
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7692282795906067

Current batch: 800
Current benign train accuracy: 0.6640625
Current benign train loss: 1.0093878507614136

Current batch: 900
Current benign train accuracy: 0.7265625
Current benign train loss: 0.7377869486808777

Current batch: 1000
Current benign train accuracy: 0.703125
Current benign train loss: 0.826921284198761

Current batch: 1100
Current benign train accuracy: 0.796875
Current benign train loss: 0.6497030854225159

Total benign train accuarcy: 74.488
Total benign train loss: 869.8129425644875

[ Test epoch: 22 ]

Test accuarcy: 14.91
Test average loss: 0.05829474806

...


Current batch: 700
Current benign train accuracy: 0.921875
Current benign train loss: 0.29220739006996155

Current batch: 800
Current benign train accuracy: 0.890625
Current benign train loss: 0.32152068614959717

Current batch: 900
Current benign train accuracy: 0.9140625
Current benign train loss: 0.30194568634033203

Current batch: 1000
Current benign train accuracy: 0.90625
Current benign train loss: 0.32260793447494507

Current batch: 1100
Current benign train accuracy: 0.90625
Current benign train loss: 0.2986440658569336

Total benign train accuarcy: 90.06733333333334
Total benign train loss: 337.3004126995802

[ Test epoch: 72 ]

Test accuarcy: 71.51
Test average loss: 0.012267391389608383
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2506944239139557

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.21106678247451782

Current batch: 200
Current benign train accura


Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.22414182126522064

Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2718869745731354

Current batch: 300
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2565622329711914

Current batch: 400
Current benign train accuracy: 0.921875
Current benign train loss: 0.21876932680606842

Current batch: 500
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3222237825393677

Current batch: 600
Current benign train accuracy: 0.8984375
Current benign train loss: 0.30417144298553467

Current batch: 700
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2905004918575287

Current batch: 800
Current benign train accuracy: 0.9375
Current benign train loss: 0.19060388207435608

Current batch: 900
Current benign train accuracy: 0.90625
Current benign train loss: 0.24582929909229279

Current batch: 1000
Current benign tr


Current batch: 900
Current benign train accuracy: 0.890625
Current benign train loss: 0.32417353987693787

Current batch: 1000
Current benign train accuracy: 0.9375
Current benign train loss: 0.1994791179895401

Current batch: 1100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.24167490005493164

Total benign train accuarcy: 90.502
Total benign train loss: 323.7369509637356

[ Test epoch: 83 ]

Test accuarcy: 56.88
Test average loss: 0.01817268377542496
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 0.890625
Current benign train loss: 0.28483882546424866

Current batch: 100
Current benign train accuracy: 0.9375
Current benign train loss: 0.20386020839214325

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.2422291338443756

Current batch: 300
Current benign train accuracy: 0.8984375
Current benign train loss: 0.24483808875083923

Current batch: 400
Current benign train accuracy: 0.9296875


Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.29941171407699585

Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.30797821283340454

Current batch: 500
Current benign train accuracy: 0.9375
Current benign train loss: 0.20461657643318176

Current batch: 600
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2909979522228241

Current batch: 700
Current benign train accuracy: 0.875
Current benign train loss: 0.3556109666824341

Current batch: 800
Current benign train accuracy: 0.8828125
Current benign train loss: 0.36163628101348877

Current batch: 900
Current benign train accuracy: 0.8828125
Current benign train loss: 0.28216221928596497

Current batch: 1000
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2734134793281555

Current batch: 1100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09740102291107178

Total benign train accuarcy: 90.80466


Current batch: 1100
Current benign train accuracy: 0.953125
Current benign train loss: 0.17672431468963623

Total benign train accuarcy: 95.328
Total benign train loss: 169.8079288378358

[ Test epoch: 94 ]

Test accuarcy: 88.89
Test average loss: 0.0035991397552192213
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.14520004391670227

Current batch: 100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.10434764623641968

Current batch: 200
Current benign train accuracy: 0.953125
Current benign train loss: 0.16569015383720398

Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.16460950672626495

Current batch: 400
Current benign train accuracy: 0.953125
Current benign train loss: 0.11641901731491089

Current batch: 500
Current benign train accuracy: 0.9375
Current benign train loss: 0.16570182144641876

Current batch: 600
Current benign train accuracy: 0.9